# Optuna

https://habr.com/ru/articles/704432/
https://optuna.readthedocs.io

## Базовый пример: оптимизация функции

In [ ]:
!pip install optuna pandas plotly nbformat scikit-learn

In [1]:
import optuna

def objective(trial):
    x = trial.suggest_float(name="x", low=-10, high=10, log=False)
    return (x - 2) ** 2

study = optuna.create_study(study_name="experiment 1")
study.optimize(objective, n_trials=100, timeout=300)

study.best_params

/Users/maestro/Desktop/Лаборатория/SANDBOX/python__optuna/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-17 23:06:07,250] A new study created in memory with name: experiment 1
[I 2025-11-17 23:06:07,265] Trial 0 finished with value: 46.56961577074823 and parameters: {'x': 8.824193415397033}. Best is trial 0 with value: 46.56961577074823.
[I 2025-11-17 23:06:07,266] Trial 1 finished with value: 5.305444747691947 and parameters: {'x': 4.303355106728432}. Best is trial 1 with value: 5.305444747691947.
[I 2025-11-17 23:06:07,267] Trial 2 finished with value: 121.78318110137589 and parameters: {'x': -9.035541722152832}. Best is trial 1 with value: 5.305444747691947.
[I 2025-11-17 23:06:07,267] Trial 3 finished with value: 0.011677880647915053 and parameters: {'x': 2.1080642431515395}. Best i

{'x': 1.9855732935753456}

## Различные типы параметров и направление оптимизации

In [2]:
def objective(x, y, z):
    match x:
        case "A":
            return (y * z) ** 2
        case "B":
            return (y * z) ** 4

def wrapper(trial):
    x = trial.suggest_categorical(name="x", choices=["A", "B"])
    y = trial.suggest_float(name="y", low=-10, high=10, step=0.1, log=False)
    z = trial.suggest_int(name="z", low=1, high=10, step=1, log=False) 
    return objective(x, y, z)

study = optuna.create_study(study_name="experiment 2", direction="maximize")
study.optimize(wrapper, n_trials=100, timeout=300)

study.best_params, study.best_value, study.best_trial

[I 2025-11-17 23:06:09,315] A new study created in memory with name: experiment 2
[I 2025-11-17 23:06:09,319] Trial 0 finished with value: 217678.23359999977 and parameters: {'x': 'B', 'y': -2.6999999999999993, 'z': 8}. Best is trial 0 with value: 217678.23359999977.
[I 2025-11-17 23:06:09,320] Trial 1 finished with value: 14641.00000000003 and parameters: {'x': 'B', 'y': 2.200000000000001, 'z': 5}. Best is trial 0 with value: 217678.23359999977.
[I 2025-11-17 23:06:09,321] Trial 2 finished with value: 47.61000000000003 and parameters: {'x': 'A', 'y': 6.900000000000002, 'z': 1}. Best is trial 0 with value: 217678.23359999977.
[I 2025-11-17 23:06:09,323] Trial 3 finished with value: 984560.0625 and parameters: {'x': 'B', 'y': -6.3, 'z': 5}. Best is trial 3 with value: 984560.0625.
[I 2025-11-17 23:06:09,324] Trial 4 finished with value: 150.0625 and parameters: {'x': 'B', 'y': 0.5, 'z': 7}. Best is trial 3 with value: 984560.0625.
[I 2025-11-17 23:06:09,325] Trial 5 finished with value:

({'x': 'B', 'y': -10.0, 'z': 10},
 100000000.0,
 FrozenTrial(number=44, state=<TrialState.COMPLETE: 1>, values=[100000000.0], datetime_start=datetime.datetime(2025, 11, 17, 23, 6, 9, 486624), datetime_complete=datetime.datetime(2025, 11, 17, 23, 6, 9, 490212), params={'x': 'B', 'y': -10.0, 'z': 10}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'x': CategoricalDistribution(choices=('A', 'B')), 'y': FloatDistribution(high=10.0, log=False, low=-10.0, step=0.1), 'z': IntDistribution(high=10, log=False, low=1, step=1)}, trial_id=44, value=None))

## Поиск по двум критериям

In [3]:
import time

def objective(x, y, z):
    match x:
        case "A":
            return (y * z) ** 2
        case "B":
            return (y * z) ** 4

def wrapper(trial):
    x = trial.suggest_categorical(name="x", choices=["A", "B"])
    y = trial.suggest_float(name="y", low=-10, high=10, step=0.1, log=False)
    z = trial.suggest_int(name="z", low=1, high=10, step=1, log=False) 
    
    t1 = time.perf_counter()
    result = objective(x, y, z)
    t2 = time.perf_counter()

    return result, t2 - t1

study = optuna.create_study(study_name="experiment 3", directions=["maximize", "minimize"])
study.optimize(wrapper, n_trials=100, timeout=10)

# Многокритериальный подбор параметров не предполагает наличие оптимальных параметров best_params.
# RuntimeError: A single best trial cannot be retrieved from a multi-objective study.
# Consider using Study.best_trials to retrieve a list containing the best trials.

optuna.visualization.plot_pareto_front(study, target_names=["Значение функции", "Время выполнения"])

[I 2025-11-17 23:06:12,383] A new study created in memory with name: experiment 3
[I 2025-11-17 23:06:12,386] Trial 0 finished with values: [70.55999999999997, 3.1250237952917814e-06] and parameters: {'x': 'A', 'y': -2.8, 'z': 3}.
[I 2025-11-17 23:06:12,388] Trial 1 finished with values: [63912.896099999976, 1.9999861251562834e-06] and parameters: {'x': 'B', 'y': -5.3, 'z': 3}.
[I 2025-11-17 23:06:12,389] Trial 2 finished with values: [2134.4399999999996, 1.0840012691915035e-06] and parameters: {'x': 'A', 'y': 7.699999999999999, 'z': 6}.
[I 2025-11-17 23:06:12,390] Trial 3 finished with values: [456976.0, 1.4579854905605316e-06] and parameters: {'x': 'B', 'y': -6.5, 'z': 4}.
[I 2025-11-17 23:06:12,391] Trial 4 finished with values: [72339.48159999997, 8.750066626816988e-07] and parameters: {'x': 'B', 'y': 8.2, 'z': 2}.
[I 2025-11-17 23:06:12,392] Trial 5 finished with values: [93789.0625, 9.579816833138466e-07] and parameters: {'x': 'B', 'y': -3.5, 'z': 5}.
[I 2025-11-17 23:06:12,393] 

### Анализ испытаний по границе Парето

In [4]:
print(f"Число комбинаций гиперпараметров по границе Парето: {len(study.best_trials)}")

trial_least_time = min(study.best_trials, key=lambda t: t.values[1])

print("Комбинация гиперпараметров с наименьшим временем выполнения: ")
print(f"\tnumber: {trial_least_time.number}")
print(f"\tparams: {trial_least_time.params}")
print(f"\tvalues: {trial_least_time.values}")

trial_with_highest_value = max(study.best_trials, key=lambda t: t.values[1])

print("Комбинация гиперпараметров с наибольшим значением: ")
print(f"\tnumber: {trial_with_highest_value.number}")
print(f"\tparams: {trial_with_highest_value.params}")
print(f"\tvalues: {trial_with_highest_value.values}")

Число комбинаций гиперпараметров по границе Парето: 5
Комбинация гиперпараметров с наименьшим временем выполнения: 
	number: 57
	params: {'x': 'A', 'y': -1.5, 'z': 7}
	values: [110.25, 3.7500285543501377e-07]
Комбинация гиперпараметров с наибольшим значением: 
	number: 84
	params: {'x': 'B', 'y': -9.9, 'z': 10}
	values: [96059601.0, 9.999785106629133e-07]


### Анализ значимости гиперпараметров

In [5]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.values[0], target_name="???"
)

## Преобразование и сохранение результатов эксперимента

In [6]:
df = study.trials_dataframe()
df.to_csv('study.csv')

df

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_x,params_y,params_z,system_attrs_NSGAIISampler:generation,state
0,0,7.056000e+01,3.125024e-06,2025-11-17 23:06:12.385765,2025-11-17 23:06:12.386797,0 days 00:00:00.001032,A,-2.8,3,0,COMPLETE
1,1,6.391290e+04,1.999986e-06,2025-11-17 23:06:12.387745,2025-11-17 23:06:12.388262,0 days 00:00:00.000517,B,-5.3,3,0,COMPLETE
2,2,2.134440e+03,1.084001e-06,2025-11-17 23:06:12.388993,2025-11-17 23:06:12.389499,0 days 00:00:00.000506,A,7.7,6,0,COMPLETE
3,3,4.569760e+05,1.457985e-06,2025-11-17 23:06:12.390075,2025-11-17 23:06:12.390543,0 days 00:00:00.000468,B,-6.5,4,0,COMPLETE
4,4,7.233948e+04,8.750067e-07,2025-11-17 23:06:12.391155,2025-11-17 23:06:12.391511,0 days 00:00:00.000356,B,8.2,2,0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...
95,95,3.841600e+02,6.670016e-07,2025-11-17 23:06:12.458298,2025-11-17 23:06:12.458507,0 days 00:00:00.000209,A,-2.8,7,1,COMPLETE
96,96,1.833966e+06,1.125009e-06,2025-11-17 23:06:12.458721,2025-11-17 23:06:12.459298,0 days 00:00:00.000577,B,4.6,8,1,COMPLETE
97,97,1.146229e+05,1.333014e-06,2025-11-17 23:06:12.460444,2025-11-17 23:06:12.460969,0 days 00:00:00.000525,B,4.6,4,1,COMPLETE
98,98,2.313441e+06,1.165987e-06,2025-11-17 23:06:12.461348,2025-11-17 23:06:12.462332,0 days 00:00:00.000984,B,7.8,5,1,COMPLETE


In [7]:
import pandas as pd

loaded = pd.read_csv('study.csv')

loaded

,Unnamed: 0,number,values_0,values_1,datetime_start,datetime_complete,duration,params_x,params_y,params_z,system_attrs_NSGAIISampler:generation,state
0,0,0,7.056000e+01,3.125024e-06,2025-11-17 23:06:12.385765,2025-11-17 23:06:12.386797,0 days 00:00:00.001032,A,-2.8,3,0,COMPLETE
1,1,1,6.391290e+04,1.999986e-06,2025-11-17 23:06:12.387745,2025-11-17 23:06:12.388262,0 days 00:00:00.000517,B,-5.3,3,0,COMPLETE
2,2,2,2.134440e+03,1.084001e-06,2025-11-17 23:06:12.388993,2025-11-17 23:06:12.389499,0 days 00:00:00.000506,A,7.7,6,0,COMPLETE
3,3,3,4.569760e+05,1.457985e-06,2025-11-17 23:06:12.390075,2025-11-17 23:06:12.390543,0 days 00:00:00.000468,B,-6.5,4,0,COMPLETE
4,4,4,7.233948e+04,8.750067e-07,2025-11-17 23:06:12.391155,2025-11-17 23:06:12.391511,0 days 00:00:00.000356,B,8.2,2,0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,95,3.841600e+02,6.670016e-07,2025-11-17 23:06:12.458298,2025-11-17 23:06:12.458507,0 days 00:00:00.000209,A,-2.8,7,1,COMPLETE
96,96,96,1.833966e+06,1.125009e-06,2025-11-17 23:06:12.458721,2025-11-17 23:06:12.459298,0 days 00:00:00.000577,B,4.6,8,1,COMPLETE
97,97,97,1.146229e+05,1.333014e-06,2025-11-17 23:06:12.460444,2025-11-17 23:06:12.460969,0 days 00:00:00.000525,B,4.6,4,1,COMPLETE
98,98,98,2.313441e+06,1.165987e-06,2025-11-17 23:06:12.461348,2025-11-17 23:06:12.462332,0 days 00:00:00.000984,B,7.8,5,1,COMPLETE


In [9]:
import joblib

joblib.dump(study, 'experiments.pkl')
study_loaded = joblib.load('experiments.pkl')
study_loaded.trials_dataframe()

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_x,params_y,params_z,system_attrs_NSGAIISampler:generation,state
0,0,7.056000e+01,3.125024e-06,2025-11-17 23:06:12.385765,2025-11-17 23:06:12.386797,0 days 00:00:00.001032,A,-2.8,3,0,COMPLETE
1,1,6.391290e+04,1.999986e-06,2025-11-17 23:06:12.387745,2025-11-17 23:06:12.388262,0 days 00:00:00.000517,B,-5.3,3,0,COMPLETE
2,2,2.134440e+03,1.084001e-06,2025-11-17 23:06:12.388993,2025-11-17 23:06:12.389499,0 days 00:00:00.000506,A,7.7,6,0,COMPLETE
3,3,4.569760e+05,1.457985e-06,2025-11-17 23:06:12.390075,2025-11-17 23:06:12.390543,0 days 00:00:00.000468,B,-6.5,4,0,COMPLETE
4,4,7.233948e+04,8.750067e-07,2025-11-17 23:06:12.391155,2025-11-17 23:06:12.391511,0 days 00:00:00.000356,B,8.2,2,0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...
95,95,3.841600e+02,6.670016e-07,2025-11-17 23:06:12.458298,2025-11-17 23:06:12.458507,0 days 00:00:00.000209,A,-2.8,7,1,COMPLETE
96,96,1.833966e+06,1.125009e-06,2025-11-17 23:06:12.458721,2025-11-17 23:06:12.459298,0 days 00:00:00.000577,B,4.6,8,1,COMPLETE
97,97,1.146229e+05,1.333014e-06,2025-11-17 23:06:12.460444,2025-11-17 23:06:12.460969,0 days 00:00:00.000525,B,4.6,4,1,COMPLETE
98,98,2.313441e+06,1.165987e-06,2025-11-17 23:06:12.461348,2025-11-17 23:06:12.462332,0 days 00:00:00.000984,B,7.8,5,1,COMPLETE
